## 예스 24 크롤링

In [153]:
# webdriver-manager 라이브러리 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()


'/Users/admin/.wdm/drivers/chromedriver/mac64/142.0.7444.61/chromedriver-mac-arm64/chromedriver'

In [154]:
from selenium import webdriver
browser = webdriver.Chrome()

In [155]:
url = 'https://www.yes24.com/product/category/bestseller?categoryNumber=001'
browser.get(url)

In [156]:
# 1 페이지의 링크 데이터  전체 수집
from selenium.webdriver.common.by import By

### 한개의 베스트셀러 링크 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')    # element 요소

### 1페이지 전체의 링크 데이터
datas = browser.find_elements(By.CLASS_NAME, 'gd_name')     # elements 리스트

for i in datas:
    print(i.get_attribute('href'))


https://www.yes24.com/product/goods/160614569
https://www.yes24.com/product/goods/153064968
https://www.yes24.com/product/goods/160408776
https://www.yes24.com/product/goods/142972546
https://www.yes24.com/product/goods/153497469
https://www.yes24.com/product/goods/148032819
https://www.yes24.com/product/goods/152645689
https://www.yes24.com/product/goods/151041569
https://www.yes24.com/product/goods/161535941
https://www.yes24.com/product/goods/149658696
https://www.yes24.com/product/goods/161341258
https://www.yes24.com/product/goods/15718330
https://www.yes24.com/product/goods/136298166
https://www.yes24.com/product/goods/159168505
https://www.yes24.com/product/goods/155113333
https://www.yes24.com/product/goods/60543924
https://www.yes24.com/product/goods/150166715
https://www.yes24.com/product/goods/149739762
https://www.yes24.com/product/goods/152298374
https://www.yes24.com/product/goods/147178677
https://www.yes24.com/product/goods/143911524
https://www.yes24.com/product/goods/

In [157]:
# 3페이지까지의 링크 데이터 전부 수집
import time

link_list = []

for i in range(1,2):
    print("*"*10, f'현재 {i}페이지 수집 중 입니다.', "*"*10)
    url = f'https://www.yes24.com/product/category/bestseller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    ### 1페이지 전체의 링크 데이터
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')     # elements 리스트

    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)

    time.sleep(3)

print(link_list)

********** 현재 1페이지 수집 중 입니다. **********
['https://www.yes24.com/product/goods/160614569', 'https://www.yes24.com/product/goods/153064968', 'https://www.yes24.com/product/goods/160408776', 'https://www.yes24.com/product/goods/142972546', 'https://www.yes24.com/product/goods/153497469', 'https://www.yes24.com/product/goods/148032819', 'https://www.yes24.com/product/goods/152645689', 'https://www.yes24.com/product/goods/151041569', 'https://www.yes24.com/product/goods/161535941', 'https://www.yes24.com/product/goods/149658696', 'https://www.yes24.com/product/goods/161341258', 'https://www.yes24.com/product/goods/15718330', 'https://www.yes24.com/product/goods/136298166', 'https://www.yes24.com/product/goods/159168505', 'https://www.yes24.com/product/goods/155113333', 'https://www.yes24.com/product/goods/60543924', 'https://www.yes24.com/product/goods/150166715', 'https://www.yes24.com/product/goods/149739762', 'https://www.yes24.com/product/goods/152298374', 'https://www.yes24.com/product

In [164]:
len(link_list)

24

In [159]:
# for link in link_list:
#     # 상세 페이지로 이동
#     browser.get(link)

browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" ")[-1]
ranking_title = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" ")[1]

title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_title

('플스포의 메타인지 투자법',
 '플스포(김동호) 저',
 '한국경제신문',
 '2025년 11월 10일',
 '9.9',
 '28',
 '90,420',
 '22,000',
 '1위',
 '국내도서')

In [165]:
# 데이터 베이스 연동 후  -> 수집한 데이터를 db에 저장 (csv)
from turtle import st
import pymysql  # pip3 install pymysql
import time
import re
from datetime import datetime

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='20241013',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:

    for i in link_list:
        browser.get(i)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2023-01-01"

#  ------------------------------------------------------------------- 별점 rating
        try:
            # 별점 있는 경우
            rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
        except:
            try:
                # 별점 없는 경우 (gd_reviewCount만 있음)
                rating = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').text
            except:
                rating = '0'

        # 🔸 문자열 정리 및 예외 처리
        rating = rating.strip()  # 공백 제거
        if not rating or rating == '-':  # 빈 값이거나 대시(-)면
            rating = '0'

        # 🔸 숫자 변환 (소수점 포함 가능)
        try:
            rating = float(rating.replace(",", ""))  # "4.5" → 4.5
        except ValueError:
            rating = 0.0
#  ------------------------------------------------------------------- 리뷰 review

        try:
            # 리뷰 수가 있는 경우
            review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
        except:
            try:
                # 리뷰 없는 경우 (gd_reviewCount 클래스)
                review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').text
            except:
                review = '0'

        # 🔸 문자열 정리
        review = review.strip().replace(",", "")

        # 🔸 숫자 아닌 텍스트 처리
        if not review.isdigit():
            review = '0'

        # 🔸 정수 변환
        review = int(review)
#  -------------------------------------------------------------------

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",", ""))

        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",", ""))

        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        part = full_text.split(" ")
        if len(part) == 1:
            ranking = 0
            ranking_title = ""
        else:
            try:
                ranking_title = part[1]
                ranking = ''.join(filter(str.isdigit, part[2]))
            except:
                ranking = 0
                ranking_title = ""

        sql = """INSERT INTO Books(
                title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_title
            )
            VALUES(
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """
        

        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_title))
        conn.commit()
        
        time.sleep(3)


In [ ]:
# 상세페이지 이동 후 데이터 크롤링
